In [2]:
# %tensorboard 

In [3]:
# for collab

import tensorflow as tf

def colab_connect():

    if tf.__version__!='2.13.0':
        %pip install tensorflow==2.13.0
        print('Stopping RUNTIME! Please run again.')
        import os
        os.kill(os.getpid(), 9)

    from google.colab import drive
    drive.mount('/content/drive/',force_remount=True)
    %cd /content/drive/My Drive/LOB/

    %pip install automodinit

try:
    import google.colab
    IS_COLAB = True
    colab_connect()
except:
    IS_COLAB =False


In [4]:
import numpy as np
from tqdm import tqdm
import keras

import data
from utilites import DataClass

seq_len = 100

In [5]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())
data.inspect_datas(row_data)

    Datas:
train     : x= (203799, 40)    | y= (203799,)      
val       : x= (50949, 40)     | y= (50949,)       
test      : x= (139586, 40)    | y= (139586,)      


In [ ]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

# Test model

In [1]:
from models import m_preln as test_model

In [6]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datasets:
train     : [398, 100, 40]
val       : [100, 100, 40]
test      : [273, 100, 40]


In [7]:
# Build
model_name = 'base_model_low'
pars = DataClass(test_model.PARAMETRS)
model = test_model.build_model(**pars.Info_expanded)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 40)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 14)           1134      
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_2 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_3 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 14)           406       
                                                                 
 layer_normalization (Layer  (None, 100, 14)           28    

In [8]:
# Callbacks

class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        f"Temp/callbacks/{model_name}",
        histogram_freq=1,
        update_freq=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"Temp/callbacks/{model_name}/checkPoint",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode="auto",
        save_freq=50,
        options=None,
        initial_value_threshold=None,
    )
]
if IS_COLAB: callbacks +[SaveCallback()]


In [9]:
# Train
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=callbacks,
)

Epoch 1/20
 10/398 [..............................] - ETA: 6:30 - loss: 2.4645 - accurancy: 0.5037 - sparce_accurancy: 0.3893

KeyboardInterrupt: 

# Search

In [ ]:
from keras_tuner.tuners import GridSearch
import keras_tuner

import models

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

In [ ]:
# Tuner parametrs
def configure(hp: keras_tuner.HyperParameters):

    class CN_search(CN_default):
        dilation_steps = hp.Int(
            'cn_layers',
            default=4,
            min_value=3,
            max_value=5,
            step=1,
        )

    class AN_search(AN_default):
        attention_heads = 3
        blocks = 2
        share_weights = hp.Boolean(
            'share_weights',
            default=True,
        )

    class FF_search(FF_default):
        dropout_rate = hp.Float(
            'dropout_rate',
            default=0.1,
            min_value=0,
            max_value=0.5,
            step=0.1,
        )

    lr = hp.Float(
        'lr',
        default=1e-4,
        min_value=1e-6,
        max_value=1e-3,
    )
    beta1 = hp.Float(
        'beta1',
        default=0.9,
        min_value=0.5,
        max_value=1.1,
        step=0.1,
    )
    beta2 = hp.Float(
        'beta1',
        default=0.999,
        min_value=0.5,
        max_value=1.1,
        sampling='log',
    )

    class Full_search(Full_default):
        cn = CN_search()
        an = AN_search()
        ff = FF_search()
        optimizer = keras.optimizers.Adam(
            name="Adam",
            learning_rate=lr,
            beta_1=beta1,
            beta_2=beta2,
        )

    return Full_search()


a = configure(keras_tuner.HyperParameters())
a.Info_expanded

In [ ]:
models.m_base.build_model(**a.Info_expanded)

In [ ]:
# def configure(hp: keras_tuner.HyperParameters):

#     class CN_search(DataClass):
#         dilation_steps = hp.Int(
#             'cn_layers',
#             default=4,
#             min_value=3,
#             max_value=5,
#             step=1,
#         )

#     class AN_search(DataClass):
#         attention_heads = 3
#         blocks = 2
#         share_weights = hp.Boolean(
#             'share_weights',
#             default=True,
#         )

#     class FF_search(DataClass):
#         dropout_rate = hp.Float(
#             'dropout_rate',
#             default=0.1,
#             min_value=0,
#             max_value=0.5,
#             step=0.1,
#         )

#     lr = hp.Float(
#         'lr',
#         default=1e-4,
#         min_value=1e-6,
#         max_value=1e-3,
#     )
#     beta1 = hp.Float(
#         'beta1',
#         default=0.9,
#         min_value=0.5,
#         max_value=1.1,
#         step=0.1,
#     )
#     beta2 = hp.Float(
#         'beta1',
#         default=0.999,
#         min_value=0.5,
#         max_value=1.1,
#         sampling='log',
#     )

#     class Full_search(DataClass):
#         cn = CN_search()
#         an = AN_search()
#         ff = FF_search()
#         optimizer = keras.optimizers.Adam(
#             name="Adam",
#             learning_rate=lr,
#             beta_1=beta1,
#             beta_2=beta2,
#         )

#     return Full_search()


In [ ]:
# Build model
model_name = 'search_test'
def search_base_model(hp):
    a = configure(hp)
    model = models.m_base.build_model(**a.Info_expanded)
    return model


In [ ]:
# Build tuner
tuner = keras_tuner.GridSearch(
    hypermodel=search_base_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    directory="search",
    project_name="base",
)

In [ ]:
# Callbacks
class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        f"Temp/callbacks/{model_name}",
        histogram_freq=1,
        update_freq=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"Temp/callbacks/{model_name}/checkPoint",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode="auto",
        save_freq=50,
        options=None,
        initial_value_threshold=None,
    )
]
if IS_COLAB: callbacks +[SaveCallback()]

In [ ]:
# Train
tuner.search(
    ds_train,
    epochs=2,
    callbacks = [
        callbacks
    ]
)
